In [53]:
import re
import jsonlines
from bs4 import BeautifulSoup

In [54]:
# Read course-code mapping
course_code_mapping = {}

with jsonlines.open("../data/course-code.jsonlines", "r") as jsonl_f:
    for obj in jsonl_f:
        title = obj["title"]
        code = obj["code"]

        course_code_mapping[title] = code


# Sort course-code mapping by title length
course_code_mapping = dict(
    sorted(course_code_mapping.items(), key=lambda x: len(x[0]), reverse=True)
)

course_code_mapping["Software Engineering for Software Engineers"] = "ENSF"

course_code_mapping

{'International Foundations Program Engineering': 'IFPE',
 'Organizational Behaviour and Human Resources': 'OBHR',
 'Human Resources and Organizational Dynamics': 'HROD',
 'International Foundations Program Business': 'IFPB',
 'Cellular, Molecular and Microbial Biology': 'CMMB',
 'School of Creative and Performing Arts': 'SCPA',
 'English For Academic Purposes Program': 'EAPP',
 'Architecture, Planning and Landscape': 'APLA',
 'Arabic Language and Muslim Cultures': 'ALMC',
 'Energy and Environment, Engineering': 'ENEE',
 'Software Engineering for Engineers': 'ENSF',
 'Interprofessional Health Education': 'IPHE',
 'Central and East European Studies': 'CEST',
 'Environmental Design Architecture': 'EVDA',
 'International Foundations Program': 'IFPX',
 'Campus Alberta Applied Psychology': 'CAAP',
 'Information Security and Privacy': 'ISEC',
 'Energy and Environmental Systems': 'EESS',
 'Communication and Media Studies': 'COMS',
 'Entrepreneurship and Innovation': 'ENTI',
 'Sustainable Syst

In [55]:
def replace_course_code(text):
    for title, code in course_code_mapping.items():
        text = re.sub(rf"{title} (\d\d\d)", f"{code} \\1", text)

    return text

replace_course_code("Computer Science 101")

'CPSC 101'

In [56]:
with jsonlines.open('../data/courses.jsonl', 'r') as jsonl_f:
    lst = [obj for obj in jsonl_f]
    
    prereqs = []
    antireqs = []
    coreqs = []

    for obj in lst:
        
        # Prerequisites
        prereq = obj["prereq"]
        if prereq: 
            prereq = BeautifulSoup(prereq).get_text()
            prereq = replace_course_code(prereq)
            prereqs.append(prereq)
            
        # Anti-requisite
        antireq = obj["antireq"]
        if antireq: 
            antireq = BeautifulSoup(antireq).get_text()
            antireq = replace_course_code(antireq)
            antireqs.append(antireq)
            
        # Corequisite
        coreq = obj["coreq"]
        if coreq: 
            coreq = BeautifulSoup(coreq).get_text()
            coreq = replace_course_code(coreq)
            coreqs.append(coreq)


In [57]:
# Write to file
with open('data/prereqs.txt', 'w') as f:
    for item in prereqs:
        f.write(item + "\n")
        
with open('data/antireqs.txt', 'w') as f:
    for item in antireqs:
        f.write(item + "\n")
        
with open('data/coreqs.txt', 'w') as f:
    for item in coreqs:
        f.write(item + "\n")


In [58]:
with jsonlines.open("../data/courses.jsonl", "r") as jsonl_f:
    lst = [obj for obj in jsonl_f]

    for obj in lst:
        # Prerequisites
        prereq = obj["prereq"]
        if prereq:
            prereq = replace_course_code(prereq)
            obj["prereq"] = prereq

        # Anti-requisite
        antireq = obj["antireq"]
        if antireq:
            antireq = replace_course_code(antireq)
            obj["antireq"] = antireq

        # Corequisite
        coreq = obj["coreq"]
        if coreq:
            coreq = replace_course_code(coreq)
            obj["coreq"] = coreq

In [59]:
with jsonlines.open("data/course-info.jsonl", "w") as jsonl_f:
    for obj in lst:
        jsonl_f.write(obj)